In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import os
import sys
import copy

import numpy as np
from scipy import ndimage
from scipy.spatial import cKDTree

import astropy.units as u
from astropy import wcs
from astropy.io import fits
from astropy.table import Table, Column
from astropy.coordinates import SkyCoord
from astropy.visualization import make_lupton_rgb
from astropy.utils.data import download_file, clear_download_cache

import matplotlib.pyplot as plt
from matplotlib import colors

In [2]:
from caterpillar import catalog
from caterpillar import utils

In [3]:
merian_dir = '/Users/song/Downloads/merian/'

cat_g09 = Table.read(os.path.join(merian_dir, 'catalog', 'Broadcut_GAMA09H.fits'))

In [4]:
cat_g09 = catalog.moments_to_shape(cat_g09, shape_type='i_sdss_shape', update=True)
cat_g09 = catalog.moments_to_shape(cat_g09, shape_type='i_cmodel_exp_ellipse', update=True)
cat_g09 = catalog.moments_to_shape(cat_g09, shape_type='i_cmodel_ellipse', update=True)
cat_g09 = catalog.moments_to_shape(cat_g09, shape_type='r_cmodel_exp_ellipse', update=True)
cat_g09 = catalog.moments_to_shape(cat_g09, shape_type='r_cmodel_ellipse', update=True)

In [56]:
def catalog_self_match(cat, ra='ra', dec='dec', rmax=3.0):
    """Internal match and label the groups"""
    # Convert (RA, Dec) into (X, Y, Z) coordinates
    ra, dec = np.asarray(cat[ra]), np.asarray(cat[dec])
    x, y, z = utils.spherical_to_cartesian(ra, dec)

    # Build a kdTree for the catalog
    kdtree_cat = cKDTree(np.column_stack([x, y, z]), leafsize=1000)
    
    # Maximum matching radius in 3-D, convert the rmax into degree
    rmax_3d = np.sqrt(2 - 2 * np.cos(np.deg2rad(rmax / 3600.)))
    
    # Internal match through kdTree query
    idxs = kdtree_cat.query_ball_tree(kdtree_cat, rmax_3d)
    
    # Keep the internally matched groups
    groups = [dd for dd in idxs if len(dd) > 1]
    
    # Remove the repeated groups and sort it based on number of elements
    group_uniq = []
    _ = [group_uniq.append(g) for g in groups if g not in group_uniq] 
    group_uniq.sort(key=len)
    
    # Setup a group ID array
    group_id = np.full(len(cat), 0, dtype=int)
    group_seq = np.full(len(cat), 1, dtype=int)

    group_n = 1
    for dd in group_uniq:
        if len(dd) > 1:
            group_id[dd] = group_n
            group_n += 1
            for ii, xx in enumerate(dd):
                group_seq[xx] = ii + 1
    
    return group_id, group_seq

In [54]:
cat = cat_g09[:1000]
rmax = 3.0 # arcsec

In [57]:
g_id, g_seq = catalog_self_match(cat_g09, rmax=3.0)

KeyboardInterrupt: 

In [6]:
ra, dec = np.asarray(cat['ra']), np.asarray(cat['dec'])

x, y, z = utils.spherical_to_cartesian(ra, dec)

kdtree_cat = cKDTree(np.column_stack([x, y, z]), leafsize=1000)

In [8]:
rmax_3d = np.sqrt(2 - 2 * np.cos(np.deg2rad(rmax / 3600.)))

In [10]:
idxs = kdtree_cat.query_ball_tree(kdtree_cat, rmax_3d)

In [33]:
groups = [dd for dd in idxs if len(dd) > 1]

In [38]:
group_uniq = []
_ = [group_uniq.append(g) for g in groups if g not in group_uniq] 

group_uniq.sort(key=len)

In [49]:
group_id = np.full(len(cat), 0, dtype=int)

group_n = 1
for dd in group_uniq:
    if len(dd) > 1:
        group_id[dd] = group_n
        group_n += 1

In [50]:
group_id

array([ 0,  0,  0,  0,  0,  0,  0,  0,  1,  1,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  2,  2,  0,  0,  0,  0,  0,  0,  3,  3,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  4,  4,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  5,  5,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  6,  6,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  7,  7,  0,  0,  0,  0,  0,  8,  8,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  9,  9,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0